In [1]:
import torch
import torch.nn as nn

### 下面验证 transpose 和 permute 的用法

In [2]:
x= torch.randn(12).reshape(3,2,2)
x

tensor([[[-0.4256, -1.3223],
         [ 0.3314, -0.6589]],

        [[-0.1577, -0.4315],
         [-0.8734, -0.0595]],

        [[-1.6218, -0.4744],
         [ 0.4396, -0.2176]]])

In [3]:
x_t = x.transpose(2,1)
x_t

tensor([[[-0.4256,  0.3314],
         [-1.3223, -0.6589]],

        [[-0.1577, -0.8734],
         [-0.4315, -0.0595]],

        [[-1.6218,  0.4396],
         [-0.4744, -0.2176]]])

In [4]:
x_t = x.permute(0,2,1)
x_t

tensor([[[-0.4256,  0.3314],
         [-1.3223, -0.6589]],

        [[-0.1577, -0.8734],
         [-0.4315, -0.0595]],

        [[-1.6218,  0.4396],
         [-0.4744, -0.2176]]])

### 多头注意力的实验

In [5]:
batch_size = 3
dim_embedding = 16
num_heads = 8
dim_qk = 16
dim_v =16
sequence_length = 4

In [6]:
x = torch.randn(batch_size,sequence_length,dim_embedding)
x.shape

torch.Size([3, 4, 16])

In [7]:
linear_q = nn.Linear(dim_embedding,dim_qk, bias=False)
linear_k = nn.Linear(dim_embedding,dim_qk, bias=False)
linear_v = nn.Linear(dim_embedding,dim_v, bias = False)

normal_factor = ( dim_qk // num_heads) ** -0.5 

In [8]:
div_q = dim_qk // num_heads
div_v = dim_v // num_heads
div_q, div_v

(2, 2)

In [9]:
q= linear_q(x).view(batch_size,sequence_length,num_heads, div_q).permute(0,2,1,3)

q.shape

torch.Size([3, 8, 4, 2])

In [10]:
k= linear_k(x).view(batch_size,sequence_length,num_heads, div_q).permute(0,2,1,3)
k.shape

torch.Size([3, 8, 4, 2])

In [11]:
v= linear_v(x).view(batch_size,sequence_length,num_heads,div_v).permute(0,2,1,3)
v.shape

torch.Size([3, 8, 4, 2])

In [12]:
dist = torch.matmul(q,k.permute(0,1,3,2)) ** normal_factor
dist.shape

torch.Size([3, 8, 4, 4])

In [31]:
atten_socre  = torch.matmul(dist, v) 
atten_socre.shape

torch.Size([3, 8, 4, 2])

In [32]:
atten_socre = atten_socre.permute(0,2,1,3)

atten_socre.shape

torch.Size([3, 4, 8, 2])

In [33]:
atten_socre = atten_socre.contiguous().view(batch_size,sequence_length,dim_v)
atten_socre.shape

torch.Size([3, 4, 16])